# Exploring Symbolic Attention

In [1]:
import torch
import torchinfo
from mechanistic_forward import symbolic_attn_forward_get_weights, block_forward_get_weights, datlm_forward_w_intermediate_results

import plotly.express as px
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

import sys; sys.path.append('..')
from dual_attention_transformer import DualAttnTransformerLM
from language_models import TransformerLM

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
def load_from_ckpt(ckpt_path):
    ckpt = torch.load(ckpt_path)
    model_config = ckpt['config']

    model_state_dict = ckpt['model']
    model_state_dict = {k.replace('_orig_mod.', ''): v for k, v in model_state_dict.items()}

    if 'n_heads_ra' in model_config:
        model = DualAttnTransformerLM(**model_config)
    else:
        model = TransformerLM(**model_config)

    model.load_state_dict(model_state_dict)

    return model

In [3]:
# model_path = '../log/DAT-ra8sa8nr32-ns512sh1-368M_2024_07_15_22_00_26/model_05000.pt'
model_path = '../log/DAT-ra8sa8nr32-368M_2024_07_11_18_17_07/model_10000.pt'
# model_path = '../log/DAT-sa16ra16nr64-1.3B_2024_07_13_21_59_55/model_05000.pt'

model_dat = load_from_ckpt(model_path).to('cuda')
model_dat = model_dat.eval()

In [4]:
torchinfo.summary(model_dat)
# torchinfo.summary(model_dat, input_data=torch.zeros((1, 1024), dtype=torch.int32).cuda(), device='cuda')

Layer (type:depth-idx)                                  Param #
DualAttnTransformerLM                                   --
├─ModuleDict: 1-1                                       --
│    └─Embedding: 2-1                                   51,511,296
│    └─Dropout: 2-2                                     --
│    └─ModuleList: 2-3                                  --
│    │    └─SymbolicAttention: 3-1                      3,146,752
│    │    └─SymbolicAttention: 3-2                      3,146,752
│    │    └─SymbolicAttention: 3-3                      3,146,752
│    │    └─SymbolicAttention: 3-4                      3,146,752
│    │    └─SymbolicAttention: 3-5                      3,146,752
│    │    └─SymbolicAttention: 3-6                      3,146,752
│    │    └─SymbolicAttention: 3-7                      3,146,752
│    │    └─SymbolicAttention: 3-8                      3,146,752
│    │    └─SymbolicAttention: 3-9                      3,146,752
│    │    └─SymbolicAttention: 3-10    

In [5]:
symbol_retriever = model_dat.layers.symbol_retriever
n_heads = symbol_retriever.n_heads
n_symbols = symbol_retriever.n_symbols
symbol_library = symbol_retriever.symbol_library.data
embedding_layer = model_dat.layers.token_embedder

AttributeError: 'ModuleDict' object has no attribute 'symbol_retriever'

In [ ]:
vocab_size = 50257
embs = embedding_layer(torch.arange(vocab_size).to('cuda').unsqueeze(0))
symbs = symbol_retriever(embs)

symbs2, symbol_attn_weights = symbolic_attn_forward_get_weights(symbol_retriever, embs)

In [ ]:
def compute_summary_stats_attn_weights(attn_weights, symbs):
    '''compute summary statistics of attention weights'''

    if type(attn_weights) == torch.Tensor:
        attn_weights = attn_weights.cpu().detach().numpy()

    attn_max_weight = attn_weights.max(axis=-1)
    attn_min_weight = attn_weights.min(axis=-1)
    attn_mean_weight = attn_weights.mean(axis=-1)

    attn_argmax = attn_weights.argmax(axis=-1)

    attn_entropy = -(attn_weights * np.log(attn_weights)).sum(axis=-1)
    unif_entropy = np.log(attn_weights.shape[-1])

    token_symbol_similarity = torch.matmul(symbs, symbs.transpose(1, 2))
    similarity_sample = token_symbol_similarity.flatten()[np.random.choice(token_symbol_similarity.numel(), 1000)].cpu().detach().numpy()

    symbol_similarity = torch.matmul(symbol_retriever.symbol_library, symbol_retriever.symbol_library.transpose(0, 1)).cpu().detach().numpy()

    attn_stats = dict(
        attn_max_weight=attn_max_weight,
        attn_min_weight=attn_min_weight,
        attn_mean_weight=attn_mean_weight,
        attn_entropy=attn_entropy,
        attn_argmax=attn_argmax,
        unif_entropy=unif_entropy,
        similarity_sample=similarity_sample,
        symbol_similarity=symbol_similarity
    )

    return attn_stats

def plot_symb_attn_stats(symb_attn_stats, n_heads, n_symbols):

    figs, axs = plt.subplots(nrows=n_heads, ncols=4, figsize=(20, 5 * n_heads))
    if np.shape(axs) == (4,):
        axs = axs.reshape(1, 4)

    for i in range(n_heads):
        sns.histplot(symb_attn_stats['attn_max_weight'][i], ax=axs[i, 0])
        axs[i, 0].axvline(1 / n_symbols, color='red', linestyle='--')
        axs[i, 0].set_title(f'Max Attention Weight (Head {i})')

        sns.histplot(symb_attn_stats['attn_argmax'][i], ax=axs[i, 1])
        axs[i, 1].set_title(f'Argmax Attention Weight (Head {i})')

        sns.histplot(symb_attn_stats['attn_entropy'][i], ax=axs[i, 2])
        axs[i, 2].axvline(symb_attn_stats['unif_entropy'], color='red', linestyle='--')
        axs[i, 2].set_title(f'Entropy of Attention Weights (Head {i})')

        sns.histplot(symb_attn_stats['similarity_sample'], ax=axs[i, 3])
        axs[i, 3].set_title(f'Similarity between Tokens (Head {i})')

    plt.tight_layout()
    plt.show()

## Layer 0: Symbol Assignment of Tokens

In [ ]:
print(f'# of symbols: {n_symbols}')
print(f'# of symbolic attention heads: {n_heads}')
print(f'# of tokens: {vocab_size}')
print(f'd_model: {model_dat.d_model}')

In [ ]:
# layer 1 (independently process token)

symb_attn_stats = compute_summary_stats_attn_weights(symbol_attn_weights[0], symbs)

plot_symb_attn_stats(symb_attn_stats, n_heads=n_heads, n_symbols=n_symbols)

## Layerwise Visualization

In [ ]:
import tiktoken

enc = tiktoken.get_encoding("gpt2")

In [ ]:
string = """
A finite-state machine (FSM) or finite-state automaton (FSA, plural: automata), finite automaton, or simply a state machine, is a mathematical model of computation. It is an abstract machine that can be in exactly one of a finite number of states at any given time. The FSM can change from one state to another in response to some inputs; the change from one state to another is called a transition. An FSM is defined by a list of its states, its initial state, and the inputs that trigger each transition. Finite-state machines are of two types—deterministic finite-state machines and non-deterministic finite-state machines. For any non-deterministic finite-state machine, an equivalent deterministic one can be constructed.

The behavior of state machines can be observed in many devices in modern society that perform a predetermined sequence of actions depending on a sequence of events with which they are presented. Simple examples are: vending machines, which dispense products when the proper combination of coins is deposited; elevators, whose sequence of stops is determined by the floors requested by riders; traffic lights, which change sequence when cars are waiting; combination locks, which require the input of a sequence of numbers in the proper order.

The finite-state machine has less computational power than some other models of computation such as the Turing machine. The computational power distinction means there are computational tasks that a Turing machine can do but an FSM cannot. This is because an FSM's memory is limited by the number of states it has. A finite-state machine has the same computational power as a Turing machine that is restricted such that its head may only perform "read" operations, and always has to move from left to right. FSMs are studied in the more general field of automata theory.
"""

tokens = torch.tensor(enc.encode(string)).unsqueeze(0).to('cuda')
tokens.shape

In [ ]:
# test_input = torch.arange(0, 512).cuda().unsqueeze(0)
logits, intermediate_results = datlm_forward_w_intermediate_results(model_dat, tokens)
print((model_dat(tokens)[0] - logits).abs().max()) # check that datlm_forward_w_intermediate_results produces the same result as model_dat


In [ ]:
# layer 1
symb_attn_stats = compute_summary_stats_attn_weights(intermediate_results['symbol_attn_scores'][0][0], symbs)

plot_symb_attn_stats(symb_attn_stats, n_heads=n_heads, n_symbols=n_symbols)

In [ ]:
# layer 2
symb_attn_stats = compute_summary_stats_attn_weights(intermediate_results['symbol_attn_scores'][1][0], symbs)

plot_symb_attn_stats(symb_attn_stats, n_heads=n_heads, n_symbols=n_symbols)

In [ ]:
# layer 3
symb_attn_stats = compute_summary_stats_attn_weights(intermediate_results['symbol_attn_scores'][2][0], symbs)

plot_symb_attn_stats(symb_attn_stats, n_heads=n_heads, n_symbols=n_symbols)

In [ ]:
# layer 4
symb_attn_stats = compute_summary_stats_attn_weights(intermediate_results['symbol_attn_scores'][3][0], symbs)

plot_symb_attn_stats(symb_attn_stats, n_heads=n_heads, n_symbols=n_symbols)

In [ ]:
# layer 5
symb_attn_stats = compute_summary_stats_attn_weights(intermediate_results['symbol_attn_scores'][4][0], symbs)

plot_symb_attn_stats(symb_attn_stats, n_heads=n_heads, n_symbols=n_symbols)

In [ ]:
# layer 6
symb_attn_stats = compute_summary_stats_attn_weights(intermediate_results['symbol_attn_scores'][5][0], symbs)

plot_symb_attn_stats(symb_attn_stats, n_heads=n_heads, n_symbols=n_symbols)

## Summary Statistic of Symbolic Attention Across Layers

In [ ]:
entropy_by_layer = []
max_attn_by_layer = []
for l in range(model_dat.n_layers):
    symb_attn_stats = compute_summary_stats_attn_weights(intermediate_results['symbol_attn_scores'][l][0], symbs)
    entropy_by_layer.append(symb_attn_stats['attn_entropy'].mean())
    max_attn_by_layer.append(symb_attn_stats['attn_max_weight'].mean())

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
sns.lineplot(x=np.arange(model_dat.n_layers), y=entropy_by_layer, ax=ax1)
ax1.set_title('Mean Attention Entropy by Layer');

sns.lineplot(x=np.arange(model_dat.n_layers), y=max_attn_by_layer, ax=ax2)
ax2.set_title('Mean Max Attention Weight by Layer');

# Questions

- What if symbol retriever is not shared across layers? Is this a limitation? TODO: implement symbol retriever for each layer. (e.g., with MQA to reduce param count and make comparable; or make symbols non-trainable, but keys trainable; or make symbols trainable but shared across layers; etc)
- Add entropy regularization to symbolic attention weights? Some other way to promot sparseness of 
- Add param count that takes weight-tying into consideration. (e.g., weight-tying embedding-unembedding or symbols across symbol-retrievers)